<a href="https://colab.research.google.com/github/agetmanova/Translator-EDTL-LTL-Diplom/blob/main/Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def disSimpl(a, b):
  if (a == True) or (b == True): return True;
  elif (a == False): return b;
  elif (b == False): return a;
  #a ∨ a = a
  elif ((type(a) != type(True)) and (type(b) != type(True)) and (a == b)):
    rules[0][1]+=1;
    return a;
  #a ∨ (b ∧ (a ∨ c)) = a ∨ (b ∧ c)
  elif (b.find('∧ ('+a+' ∨ ') >= 0):
    b2 = b.replace('('+a+' ∨ ', '');
    b2 = b2.replace( ')', '', 1);
    rules[1][1]+=1;
    return '(' +a+ ' ∨ ' +b2+ ')';
  #a ∨ F(a ∨ b) = F(a ∨ b)
  elif (b.find('F('+a+' ∨ ') >= 0): 
    rules[2][1]+=1;
    return b;
  #a ∨ (b U a) = (b U a)
  elif ((b[0:1] == '(') and (b.find (' U '+a+')') >= 0)): 
    rules[3][1]+=1;
    return b;
  #G(¬a) ∨ F(a) = true
  elif ((a[0:3] == 'G(¬') and (b[0:2] == 'F(') and (a[3:] == b[2:])): 
    rules[5][1]+=1;
    return True;
  #a ∨ F(a) = F(a)
  elif ((b[0:2] == 'F(') and (b[2:-1] == a)): 
    rules[6][1]+=1;
    return b;
  elif (b.find(' ∧ '+a+')') > 0): 
    rules[8][1]+=1;
    return a;
  #a ∨ (a ∨ b) = a V b | a ∨ (b ∨ a) = a ∨ b
  elif (b.find('('+a+' ∨ ') == 0): 
    rules[9][1]+=1;
    return b;

  #G(¬a) ∨ (F(a) ∨ (с)) = (с)
  elif ((a[0:3] == 'G(¬') and (b[0:3] == '(F(') and (a[3:-1] == b[3:len(a[3:-1])+3]) and (b.find(') ∨ (') == len(a[3:-1])+3)): 
    rules[41][1]+=1;
    return b[len(a[3:-1])+7:-1];

  #a ∨ (b U (a ∨ c)) = (b U (a ∨ c))
  elif (b.find(' U (' + a + ' ∨ ') > 0):
    rules[43][1] +=1;
    return b;

  else: 
    return '(' +a+ ' ∨ ' +b+ ')'

In [ ]:
def conSimpl(a, b):
  if (a == False) or (b == False): return False;
  elif (a == True): return b;
  elif (b == True): return a;
  
  else: return '('+a+' ∧ '+b+')'

In [ ]:
def no(a):
  if (type(a) == type(True)): return not a;
  
  else: return "¬" + a;

In [ ]:
def implSimpl(a, b):
  if ((type(a) == type(True)) or (type(b) == type(True))): return disSimpl(no(a),b);

  else:  return '('+a+' → '+b+')'

In [ ]:
def FutureSimpl(a):
  if (type(a) == type(True)): return a;
  elif (a[0] != '('): return 'F('+a+')';
  else: return 'F'+a+''

In [ ]:
def GloballySimpl(a):
  if (type(a) == type(True)): return a;
  #G(F(a)) = GF(a)
  elif ((a[0] == 'F') and (a[1] == '(') and (a[len(a)-1] == ')')): 
    rules[32][1]+=1;
    return 'G'+a;
  #G(G(a)) = G(a)
  elif ((a[0] == 'G')and (a[1] == '(') and (a[len(a)-1] == ')')): 
    rules[33][1]+=1;
    return a;

  #51 G(G(a ∧ ¬b) ∨ (a U (b ∧ a))) = G(a)
  elif((a[:3] == '(G(') and (a.find(' ∧ ¬') > 0) and (a.find(') ∨ (') > a.find(' ∧ ¬'))
        and (a.find(a[3:a.find(' ∧ ¬')] + ' U (') > 0)
        and (a.find(a[a.find(' ∧ ¬')+4:a.find(') ∨ (')] + ' ∧ ' + a[3:a.find(' ∧ ¬')]) > 0)):
    rules[51][1] +=1;
    return 'G(' + a[3:a.find(' ∧ ¬')] + ')';

  #52 G(G(a ∧ ¬b) ∨ ((a ∧ ¬b) U (b ∧ (a U (a ∧ c))))) = G(a ∧ (G(¬b) ∨ F(b ∧ F(c))))
  elif((a[:3] == '(G(') and (a.find(' ∧ ¬') > 0) and (a.find(') ∨ (') > a.find(' ∧ ¬'))
        and (a.find(a[3:(a.find(') ∨ ('))] + ') U (' + a[a.find(' ∧ ¬')+4:a.find(') ∨ (')] +
                    ' ∧ (' + a[3:a.find(' ∧ ¬')] + ' U (' + a[3:a.find(' ∧ ¬')] + ' ∧ ') > a.find(') ∨ (')) ):
    a1 = a[3:a.find(' ∧ ¬')];
    nb1 = a[a.find(' ∧ ¬')+3:a.find(') ∨ (')]
    b1 = a[a.find(' ∧ ¬')+4:a.find(') ∨ (')]
    
    n ='G(' + a[3:a.find(' ∧ ¬')] + ' ∧ ¬' + a[a.find(' ∧ ¬')+4:a.find(') ∨ (')] + ') ∨ ((' + a[3:(a.find(') ∨ ('))] + ') U (' + a[a.find(' ∧ ¬')+4:a.find(') ∨ (')] + ' ∧ (' + a[3:a.find(' ∧ ¬')] + ' U (' + a[3:a.find(' ∧ ¬')] + ' ∧ '
    t = a.find(n)
    tlen = len(n)
    c = a[tlen + 1:-5]

    rules[52][1] +=1;
    return 'G' + conSimpl(a1, disSimpl(GloballySimpl(nb1), FutureSimpl(disSimpl(b1, FutureSimpl(c)))))

  #53 G(G(a ∧ ¬b) ∨ ((a ∧ ¬b) U (b ∧ (a ∧ c)))) = G(a ∧ (G(¬b) ∨ F(b ∧ c)))
  elif((a[:3] == '(G(') and (a.find(' ∧ ¬') > 0) and (a.find(') ∨ (') > a.find(' ∧ ¬'))
        and (a.find(a[3:(a.find(') ∨ ('))] + ') U (' + a[a.find(' ∧ ¬')+4:a.find(') ∨ (')] +
                    ' ∧ (' + a[3:a.find(' ∧ ¬')] + ' ∧ ') > a.find(') ∨ (')) ):
    a1 = a[3:a.find(' ∧ ¬')];
    nb1 = a[a.find(' ∧ ¬')+3:a.find(') ∨ (')]
    b1 = a[a.find(' ∧ ¬')+4:a.find(') ∨ (')]

    n ='G(' + a[3:a.find(' ∧ ¬')] + ' ∧ ¬' + a[a.find(' ∧ ¬')+4:a.find(') ∨ (')] + ') ∨ ((' + a[3:(a.find(') ∨ ('))] + ') U (' + a[a.find(' ∧ ¬')+4:a.find(') ∨ (')] + ' ∧ (' + a[3:a.find(' ∧ ¬')] + ' ∧ '
    t = a.find(n)
    tlen = len(n)
    c = a[tlen + 1:-4]

    rules[53][1] +=1;
    return 'G' + conSimpl(a1, disSimpl(GloballySimpl(nb1), FutureSimpl(disSimpl(b1, c))));

  #44 G(G(a) ∨ (a U b)) = G(a ∧ F(b))
  elif ((a[0:3] == '(G(') and (a.find(') ∨ (') > 0) and (a.find(a[3:a.find(') ∨ (')] + ' U '))):
    rules[44][1] +=1;
    return GloballySimpl(conSimpl(a[3:a.find(') ∨ (')], FutureSimpl(a[a.find(' U ')+3:-2])))
  
  #45 G((a ∧ b) U (a ∧ c)) = G(a ∧ (b U c))
  elif ((a[0:2] == '((') and (a.find(' ∧ ') > 0) and (a.find(') U (') > a.find(' ∧ ')) 
          and (a[2:a.find(' ∧ ')] == a[a.find(') U (')+5:a.find(' ∧ ', a.find(' ∧ ') + 1)])):
    rules[45][1] +=1;
    b = a[a.find(' ∧ ') + 3: a.find(') U (')]
    c = a[a.find(' ∧ ', a.find(' ∧ ') + 1) + 3:-2]
    a1 = a[2:a.find(' ∧ ')]   
    return GloballySimpl(conSimpl(a1, UntilSimpl(b,c)));
    
  elif (a[0] != '('): return 'G('+a+')';
  else: return 'G'+a;

In [ ]:
def UntilSimpl(a, b):
  if (b == False): return False;
  elif (b == True): return True;
  elif ((a == False) and (type(b) != type(True))): return b;
  elif ((a == True) and (type(b) != type(True))): return FutureSimpl(b);

  #a U a = a
  elif (a == b):
    rules[34][1] +=1;
    return a;

  # ¬a U a = F(a)
  elif((a.find('¬') == 0) and (b == a[1:])):
    rules[35][1] +=1;
    return FutureSimpl(b);

  #¬a U (b ∨ a) = F(a) ∨ (¬a U b)
  elif ((a[0] == '¬') and (b.find ('∨ ' + a[1:]) > 0)):
    rules[42][1]+=1;
    b1 = b[1:(b.find ('∨ ' + a[1:]))-1];
    return disSimpl(FutureSimpl(a[1:]), UntilSimpl(a, b1));

  #46 (a ∧ b) U (c ∨ a) = a ∨ ((a ∧ b) U c)
  elif ((a.find(' ∧ ') > 0) and (b.find(' ∨ ') > 0) and (a[1:a.find(' ∧ ')] == b[b.find(' ∨ ')+3:-1])):
    rules[46][1] +=1;
    c = UntilSimpl(a, b[1:b.find(' ∨ ')]);
    return disSimpl(a[1:a.find(' ∧ ')], c);

  #47 (a ∧ b) U a = a
  elif(a.find('(' + b + ' ∧ ') == 0):
    rules[47][1] +=1;
    return b;

  #48 a U (b ∨ a) = a ∨ (a U b)
  elif (b.find(' ∨ ' + a + ')') > 0):
    rules[48][1] +=1;
    return disSimpl(a, UntilSimpl(a, b[1:b.find(' ∨ ')]));

  #50 (a ∧ ¬b) U (b ∧ a) = a U (b ∧ a)
  elif ((a.find(' ∧ ') > 0) and (b.find(' ∧ ') > 0) 
        and (a[1:a.find(' ∧ ')] == b[b.find(' ∧ ')+3:-1])
        and (a[a.find(' ∧ ')+3] == '¬') 
        and (b[1:b.find(' ∧ ')] == a[a.find(' ∧ ')+4:-1])):
    rules[50][1] +=1;
    return UntilSimpl(a[1:a.find(' ∧ ')], b)

  else: return '('+a+' U '+b+')' 

In [ ]:
def WeakUntil(a, b):
  if ((type(a) == type(True)) and (type(b) == type(True))):
    if ((a == False) and (b == False)): return False;
    else: return True;
  elif ((type(b) != type(True)) and (type(a) != type(True))): return '('+a+' W '+b+')';
  elif (type(b) != type(True)):
    if (a == False): return b;
    elif (a == True): return True;
  elif (type(a) != type(True)):
    if (b == True): return True;
    elif (b == False): return Globally(a);

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 

data_frame = pd.read_csv('atributs.csv', encoding='utf8')

data_frame

In [ ]:
def konv(a_txt):
  if (a_txt == 'true*'): return True;
  elif (a_txt == 'false*'): return False;
  else: return a_txt;

In [ ]:
def BoolToString(a):
  if (type(a) == type(True)): 
    if (a == True): return "true";
    elif (a == False): return "false";
  else: return a;

In [ ]:
a = 12
mas = [] 
for i1 in range(243): 
    mas.append([0] * a)


for i in data_frame.index:
  release_txt = data_frame['release'][i]
  delay_txt = data_frame['delay'][i]
  final_txt =  data_frame['final'][i]
  reaction_txt =  data_frame['reaction'][i]
  invariant_txt =  data_frame['invariant'][i]

  triggerVAR_txt = data_frame['trigerVAR'][i];
  triggerTrue_txt = data_frame['trigerTRUE'][i];

  release = konv(release_txt);
  delay = konv(delay_txt);
  final = konv(final_txt);
  reaction = konv(reaction_txt);
  invariant = konv(invariant_txt);

  trigger = 'trig'
  x0 = con(trigger, no(release));
  x1 = con(invariant, reaction);
  x2 = dis(release, x1);
  x3 = con(invariant, no(delay));
  x4 = Until(x3, x2);
  x5 = con(final, x4);
  x6 = dis(release, x5)
  x7 = con(invariant, no(final));
  x8 = Until(x7, x6);

  x9 = con(invariant, no(final));
  x10 = Globally(x9);
  x11 = dis(x10, x8);
  x12 = con(invariant, x11);
  x13 = impl(x0, x12);
  x14 = Globally(x13);

  
  trigger = True
  
  y0 = con(trigger, no(release));
  y1 = con(invariant, reaction);
  y2 = dis(release, y1);
  y3 = con(invariant, no(delay));
  y4 = Until(y3, y2);
  y5 = con(final, y4);
  y6 = dis(release, y5)
  y7 = con(invariant, no(final));
  y8 = Until(y7, y6);
  y9 = con(invariant, no(final));

  y10 = Globally(y9);
  y11 = dis(y10, y8);
  y12 = con(invariant, y11);
  y13 = impl(y0, y12);
  y14 = Globally(y13);

  trigger = False
  z1 = True;
  
  mas[i][0]=BoolToString(release);
  mas[i][1]=BoolToString(delay);
  mas[i][2]=BoolToString(final);
  mas[i][3]=BoolToString(reaction);
  mas[i][4]=BoolToString(invariant);

  mas[i][5]=BoolToString(x14);
  mas[i][6]=BoolToString(y14);
  mas[i][7]=BoolToString(z1);

for i in range(0, len(mas)):
    for i2 in range(0, len(mas[i])):
        print(mas[i][i2], end=' ')
    print()


from openpyxl import Workbook
wb = Workbook()
ws = wb.active


for subarray in mas:
    ws.append(subarray)
wb.save('./edtl-ltl.xlsx')